In [1]:
%pip install rapidfuzz
%pip install pandas

import pandas as pd
from rapidfuzz import process, fuzz
import re

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
companies_data = pd.read_csv("../data/companies.csv")
articles_data = pd.read_csv("../data/article_data/articles_about_breaches_with_company_name_not_gov.csv")

## Exact string match

In [4]:
companies_data['name_lower'] = companies_data['name'].str.lower()
articles_data['CompanyMentioned_lower'] = articles_data['CompanyMentioned'].str.lower()

# Perform the join on the lowercase columns
result_df = pd.merge(companies_data, articles_data, left_on='name_lower', right_on='CompanyMentioned_lower', how='inner')

# Drop the helper lowercase columns if desired
result_df = result_df.drop(columns=['name_lower', 'CompanyMentioned_lower'])

# Sort by 'name' (from companies_data)
result_df.sort_values(by=['CompanyMentioned', 'name']).drop_duplicates(subset=['Headline', 'URL'], inplace=True)

result_df.to_csv("../data/article_data/joined_articles_companies.csv", index=False)

## match on companies with dupicate names

In [ ]:
companies_data_dups = pd.read_csv("../data/article_data/duplicate_name_companies.csv")

companies_data_dups['name_lower'] = companies_data_dups['name'].str.lower()
articles_data['CompanyMentioned_lower'] = articles_data['CompanyMentioned'].str.lower()

# Perform the join on the lowercase columns
result_df = pd.merge(companies_data_dups, articles_data, left_on='name_lower', right_on='CompanyMentioned_lower', how='inner')

# Drop the helper lowercase columns if desired
result_df = result_df.drop(columns=['name_lower', 'CompanyMentioned_lower'])

# Sort by 'name' (from companies_data_dups)
result_df.sort_values(by=['name', 'total employee estimate']).drop_duplicates(subset=['Headline', 'URL'], inplace=True)

result_df.to_csv("../data/article_data/joined_articles_companies.csv", index=False)

In [3]:
## match on the duplicate company names

cd = pd.read_csv("../data/article_data/duplicate_name_companies.csv")

cd['name_lower'] = cd['name'].str.lower()
articles_data['CompanyMentioned_lower'] = articles_data['CompanyMentioned'].str.lower()

# Perform the join on the lowercase columns
result_df = pd.merge(cd, articles_data, left_on='name_lower', right_on='CompanyMentioned_lower', how='inner')

# Drop the helper lowercase columns if desired
result_df = result_df.drop(columns=['name_lower', 'CompanyMentioned_lower'])

# Sort by 'name' (from cd)
result_df.sort_values(by=['CompanyMentioned', 'name']).drop_duplicates(subset=['Headline', 'URL'], inplace=True)

result_df.to_csv("../data/article_data/joined_articles_dup_companies.csv", index=False)

remove duplicates code. keep the one with max total emplyee estimate

In [ ]:
exact_matched = pd.read_csv("../data/article_data/joined_articles_companies.csv")

# drop duplicate rows by Headline and URL
df_dedup = exact_matched.loc[exact_matched.groupby(['Headline', 'URL'])['total employee estimate'].idxmax()].reset_index(drop=True)

df_dedup.to_csv("../data/article_data/joined_articles_companies_no_duplicates.csv", index=False)

## Fuzzy matching

Problem with this is that it matches articles to multiple companies when they should only be matched to one. My best idea right now is to go through them by hand to determine which is should be matched to, but I am hesitant to do that right now because we may rerun classfier and such so would rather only do the manual work on the data we are going to use for sure.

In [6]:
companies_data_aux = companies_data.head(1000).copy()
not_matched_articles_data_aux = pd.read_csv("../data/article_data/classified_articles_not_exact_matched_to_companies_dataset.csv")

def preprocess(text):
    # Remove punctuation and convert to lowercase
    if pd.isna(text):
        return ""
    return re.sub(r'[^\w\s]', '', text).lower()

# Preprocess your data
companies_data_aux['name_processed'] = companies_data_aux['name'].apply(preprocess)
not_matched_articles_data_aux['CompanyMentioned_processed'] = not_matched_articles_data_aux['CompanyMentioned'].apply(preprocess)

# Perform the matching
def match_company(row, companies_data_aux):
    # Filter companies where the processed name starts with the processed CompanyMentioned
    return companies_data_aux[companies_data_aux['name_processed'].str.startswith(row['CompanyMentioned_processed'] + " ")]

# Apply matching to each row in articles data
matches = not_matched_articles_data_aux.apply(
    lambda row: match_company(row, companies_data_aux), axis=1
)

# Combine results into a single DataFrame
results = pd.concat([
    pd.concat([not_matched_articles_data_aux.iloc[[i]].reset_index(drop=True), match.reset_index(drop=True)], axis=1)
    for i, match in enumerate(matches) if not match.empty
], ignore_index=True)

print(len(results))


# Drop the processed columns to keep only original data
results = results.drop(columns=['name_processed', 'CompanyMentioned_processed'])

# drop duplicate rows by max total employee estimate
results = results.loc[results.groupby(['Headline', 'URL'])['total employee estimate'].idxmax()].reset_index(drop=True)

# Sort by 'name' (from companies_data)
results = results.sort_values(by='name').drop_duplicates().reset_index(drop=True)

results.to_csv("../data/article_data/fuzzy_match_non_exact_matched_articles.csv", index=False) 


861


In [14]:
companies_data_aux = companies_data.dropna(subset=['domain'], how='any').copy()
not_matched_articles_data_aux = pd.read_csv("../data/article_data/classified_articles_not_exact_matched_to_companies_dataset.csv")

def preprocess(text):
    # Remove punctuation and convert to lowercase
    if pd.isna(text):
        return ""
    return re.sub(r'[^\w\s]', '', text).lower()

# Preprocess your data
companies_data_aux['name_processed'] = companies_data_aux['name'].apply(preprocess)
not_matched_articles_data_aux['CompanyMentioned_processed'] = not_matched_articles_data_aux['CompanyMentioned'].apply(preprocess)

# Perform the matching
def match_company(row, companies_data_aux):
    # Filter companies where the processed name starts with the processed CompanyMentioned
    return companies_data_aux[companies_data_aux['domain'].str.startswith(row['CompanyMentioned_processed'] + ".")]

# Apply matching to each row in articles data
matches = not_matched_articles_data_aux.apply(
    lambda row: match_company(row, companies_data_aux), axis=1
)

# Combine results into a single DataFrame
results = pd.concat([
    pd.concat([not_matched_articles_data_aux.iloc[[i]].reset_index(drop=True), match.reset_index(drop=True)], axis=1)
    for i, match in enumerate(matches) if not match.empty
], ignore_index=True)

print(len(results))


# Drop the processed columns to keep only original data
results = results.drop(columns=['CompanyMentioned_processed', 'name_processed'])

# drop duplicate rows by max total employee estimate
results = results.loc[results.groupby(['Headline', 'URL'])['total employee estimate'].idxmax()].reset_index(drop=True)

# Sort by 'name' (from companies_data)
results = results.sort_values(by='name').drop_duplicates().reset_index(drop=True)

results.to_csv("../data/article_data/url_match_non_exact_matched_articles.csv", index=False) 

# 196 m for full table to run
# 352 m the second time


5749


As a fix to the problem above of duplicate articles we remove duplicates from the results by max 'total employee estimate'.

In [ ]:
# fuzzy_matched = pd.read_csv("../data/article_data/joined_articles_companies_fuzzy.csv")

# # drop duplicate rows by Headline and URL
# df_dedup = exact_matched.loc[exact_matched.groupby(['Headline', 'URL'])['total employee estimate'].idxmax()].reset_index(drop=True)

# fuzzy_matched.to_csv("../data/article_data/joined_articles_companies_fuzzy_no_duplicates.csv", index=False)

## Articles not matched

In [ ]:
fuzzy_matched = pd.read_csv("../data/article_data/joined_articles_companies_fuzzy_no_duplicates.csv")

classified_articles = pd.read_csv("../data/articles_about_breaches_with_company_name.csv")

matched_company_names = fuzzy_matched['CompanyMentioned'].tolist()

# the companies in the "companymentioned" column that did not get matched to a company from the company dataset during fuzzy matching
not_matched = classified_articles[~classified_articles['CompanyMentioned'].isin(matched_company_names)]

not_matched.to_csv("../data/article_data/classified_articles_not_matched_to_companies_dataset.csv", index=False)

In [2]:
## sort not exact matched articles by company mentioned column

df = pd.read_csv("../data/article_data/classified_articles_not_exact_matched_to_companies_dataset.csv")

df = df.sort_values(by='CompanyMentioned').reset_index(drop=True)

df.to_csv("../data/article_data/sorted_classified_articles_not_exact_matched_to_companies_dataset.csv", index=False)

In [3]:
import pandas as pd

# Example DataFrames
df1 = pd.DataFrame({
    'Company mentioned': ['Tech', 'Data', 'Smart'],
    'Other Info': [1, 2, 3]
})

df2 = pd.DataFrame({
    'name': ['Tech Corp', 'Data Systems', 'AI Smart Co', 'Random Name'],
    'Details': ['A', 'B', 'C', 'D']
})

# Step 1: Create a custom join condition
def starts_with_condition(row, df2):
    return df2[df2['name'].str.startswith(row['Company mentioned'])]

# Step 2: Use apply to match rows from df1 with df2
matches = df1.apply(lambda row: starts_with_condition(row, df2), axis=1)

# Step 3: Combine the results into a single DataFrame
results = pd.concat([pd.concat([df1.iloc[[i]].reset_index(drop=True), match.reset_index(drop=True)], axis=1) 
                     for i, match in enumerate(matches) if not match.empty], ignore_index=True)

print(results)


  Company mentioned  Other Info          name Details
0              Tech           1     Tech Corp       A
1              Data           2  Data Systems       B
